In [1]:
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.datasets import reuters

(train_x, train_y), (test_x, test_y) = reuters.load_data(num_words=10000)

In [3]:
def vec_seq(seqs, dim=10000):
    results = np.zeros((len(seqs), dim))
    for i, seq in enumerate(seqs):
        results[i, seq] = 1
    return results

train_x = vec_seq(train_x)
test_x = vec_seq(test_x)

In [4]:
train_y = keras.utils.to_categorical(train_y)
test_y = keras.utils.to_categorical(test_y)

In [5]:
from keras import models, layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(46, activation='softmax'))
    
    model.compile(optimizer='rmsprop',
                 loss='categorical_crossentropy',
                 metrics=['acc'])
    
    return model



In [6]:
import random

k = 4
random.shuffle(train_x)
random.shuffle(train_y)

results = [[],[]]
scale = round(len(train_x) / k)

for i in range(k):
    if i != k-1:
        val_x = train_x[i*scale:(i+1)*scale]
        val_y = train_y[i*scale:(i+1)*scale]

        partial_train_x = np.array(list(train_x[:i*scale]) + list(train_x[(i+1)*scale:]))
        partial_train_y = np.array(list(train_y[:i*scale]) + list(train_y[(i+1)*scale:]))
    else:
        val_x = train_x[i*scale:]
        val_y = train_y[i*scale:]

        partial_train_x = train_x[:i*scale]
        partial_train_y = train_y[:i*scale]
        

    model = build_model()
    history = model.fit(partial_train_x, partial_train_y,
                        epochs=10,
                        batch_size=512,
                        validation_data=[val_x,val_y]
                       )

    results[0].append(model.evaluate(test_x,test_y)[0])
    results[1].append(model.evaluate(test_x,test_y)[1])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 6736 samples, validate on 2246 samples
Epoch 1/10
6736/6736 [==============================] - 2s 335us/step - loss: 2.8859 - acc: 0.3129 - val_loss: 2.4889 - val_acc: 0.3442
Epoch 2/10
6736/6736 [==============================] - 1s 110us/step - loss: 2.3985 - acc: 0.3624 - val_loss: 2.4853 - val_acc: 0.3451
Epoch 3/10
6736/6736 [==============================] - 1s 109us/step - loss: 2.3013 - acc: 0.3799 - val_loss: 2.4108 - val_acc: 0.3232
Epoch 4/10
6736/6736 [==============================] - 1s 111us/step - loss: 2.1822 - acc: 0.4026 - val_loss: 2.4623 - val_acc: 0.3192
Epoch 5/10
6736/6736 [==============================] - 1s 108us/step - loss: 2.0785 - acc: 0.4238 - val_loss: 2.5342 - val_acc: 0.3112
Epoch 6/10
6736/6736 [==============================] - 1s 108us/step - loss: 1.9771 - acc: 0.4469 - val_loss: 2.6424 - val_acc: 0.2872
Epoch 7/10
6736/

In [7]:
print(results[0])
print(results[1])

print(sum(results[0])/k)
print(sum(results[1])/k)

[3.0795465817116034, 3.382558272337128, 2.883686419161748, 2.871297688327086]
[0.32902938562743067, 0.35663401608157136, 0.24398931436313653, 0.35574354413143777]
3.0542722403843916
0.32134906505089406
